## Enderscopy stage and lights demo

This notebook shows how to connect and drive an enderscope stage from Python.

In [ ]:
import time
from enderscope import Stage, Panel, Enderlights
from serial_utils import serial_ports

In [ ]:
# list available serial ports
ports = serial_ports()
print (ports)

In [ ]:
lightport, stageport = ports
stageport, lightport = ports

In [ ]:
# connect to enderscope and create a stage object
s = Stage(stageport, 115200)

In [ ]:
# create a stage control panel
p = Panel(s)

In [ ]:
p.recorded_positions

In [ ]:
# iterate over manually recorded positions
for i in range(5):
    for pos in p.recorded_positions:
        s.move_position(pos)
        s.finish_moves()
        

In [ ]:
# home the stage for positionning reproducibility
s.home()
# move to a safe distance along the z axis, axis is x, y or z, distances are in millimeters
s.move_axis('z',10)

In [ ]:
# read the stage position
s.get_position()

In [ ]:
# position can be retrieved as a dict
s.get_position(dict=True)

In [ ]:
# move to (x,y(,z)) absolute coordinate
s.move_absolute(50,120)

In [ ]:
# debug view the actual G-Code that was sent
s.move_absolute(20,30, debug=True)

In [ ]:
# you can head towards standard directions, distances are always > 0
s.move_towards('east',70)

In [ ]:
# you can move by some amount, relative to the current position
s.move_relative(0,0,5)

In [ ]:
for i in range(15):
    s.move_relative(-1,0,0)

In [ ]:
# a position is a tuple of length 2 or 3 to which you can move
pos = (20,20)
s.move_position(pos)

In [ ]:
p = (40,40,10)
s.move_position(p)

In [ ]:
# sending specific gcode to read device parameters
s.write_code(f"M503", debug=True)

In [ ]:
s.write_code(f"M203 Z50") # increase z axis speed to 50
s.move_axis('z',40) # go up fast
s.write_code(f"M203 Z5") # restore z axis speed to 5
s.move_axis('z',-40) # go down slowly

In [ ]:
l = Enderlights(lightport)

In [ ]:
# querrying LED state
done = l.write_code(f"?\n")

In [ ]:
# setting LED colors
l.color(100,0,0)
l.green(10)
l.blue(30)

In [ ]:
# blinking LEDs ;-)
for i in range(10):
    l.shutter(True)
    time.sleep(0.05)
    l.red(100)
    l.shutter(False)
    time.sleep(0.05)

In [ ]:
from picamera2 import Picamera2
picam2 = Picamera2()
picam2.start(show_preview=True)
# picam2.capture_file("test.jpg")
# picam2.close()

In [ ]:
# create an array of positions to perform a raster scan
# import time
import numpy as np

origin = [56,100] # coordinates of first position
steps = [25,-25] # x and y offsets to next item
grid = np.array(list((x,y) for y in range(3) for x in range(4))) * steps + origin
for p in grid:
    s.move_position(p)
    s.finish_moves() 
    l.shutter(True)
    time.sleep(0.1)
    picam2.capture_file(str(p)+"_test.jpg")
    #time.sleep(2)
    l.shutter(False)

s.move_position(origin) # move back to origin
picam2.close()

In [ ]:
# arbitrary curve from gcode
# create curve in inkscape, export as svg, import in slicer, export gcode
import os
def read_g_code(fname):
    fpath = os.path.join(".", fname)
    with open(fpath, "r") as text_file:
        data = text_file.read()
    return data
path = read_g_code("tests/path2.gcode")

In [ ]:
for i in range(1,50):
    bit = path.split('\n')[i]
    bit = bit[0:bit.index('E')-1]
    s.write_code(bit, debug=False)